# ML tests

### Libraries

In [250]:
# necessary to import other modules
import sys
sys.path.append('../../application')

from functions.mysql import *
from functions.machine_learning_model import *

import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier

print("Libraries Imported")

Libraries Imported


### MySQL Connection

In [251]:
#host = "127.0.0.1"
#port = 3306
#database = "db_bdt_project"
#user = "root"
#password = "dany1998"
#
#db_connection = connectToMySQL_Alchemy(host, port, database, user, password, False)

In [252]:
#table = "appalti_aggiudicatari"
#df = retrieveAllDataFromTable_Alchemy(table, db_connection)
#
#print(df.head())

In [253]:
#print("\nTotal rows:", df.shape[0])

Which is practically the same as

In [254]:
df = pd.read_csv("../../application/datasets/clean_data/appalti_aggiudicatari.csv", sep=";")
print("\nTotal rows:", df.shape[0])


Total rows: 1514852


### Select a smaller sample for testing

In [255]:
# ensure the data type is correct
df['data_pubblicazione'] = pd.to_datetime(df['data_pubblicazione'])  

In [256]:
# select the first 11 month of 2020
start_date = "2020-01-01"
end_date = "2020-11-30"
mask = (df['data_pubblicazione'] >= start_date) & (df['data_pubblicazione'] <= end_date)

training_df = df.loc[mask]
training_df = training_df.reset_index()
print("\nTotal rows for training:", training_df.shape[0])


Total rows for training: 110972


In [257]:
# select the December 2020 (to use as test)
start_date = "2020-12-01"
end_date = "2020-12-31"
mask = (df['data_pubblicazione'] >= start_date) & (df['data_pubblicazione'] <= end_date)

test_df = df.loc[mask]
test_df = test_df.reset_index()
print("\nTotal rows for testing:", test_df.shape[0])


Total rows for testing: 7177


## List of variables we will use

### Numeric Variables (no changes needed)
- importo_complessivo_gara
- n_lotti_componenti
- importo_lotto

### Categorical Variable (need of dummy data)
- settore
- tipo_scelta_contraente
- modalita_realizzazione
- denominazione_amministrazione_appaltante
- sezione_regionale
- descrizione_cpv
- tipo_aggiudicatario

In [258]:
def printColInfo(col):
    temp_set = set(col)
    print("Total factors:", len(temp_set))
    #print("Factors:\n", temp_set)

In [259]:
print("settore")
printColInfo(df.settore)

print("\ntipo_scelta_contraente")
printColInfo(df.tipo_scelta_contraente)

print("\nmodalita_realizzazione")
printColInfo(df.modalita_realizzazione)

print("\ndenominazione_amministrazione_appaltante")
printColInfo(df.denominazione_amministrazione_appaltante)

print("\nsezione_regionale")
printColInfo(df.sezione_regionale)

print("\ndescrizione_cpv")
printColInfo(df.descrizione_cpv)

print("\ntipo_aggiudicatario")
printColInfo(df.tipo_aggiudicatario)

settore
Total factors: 2

tipo_scelta_contraente
Total factors: 30

modalita_realizzazione
Total factors: 23

denominazione_amministrazione_appaltante
Total factors: 22627

sezione_regionale
Total factors: 25

descrizione_cpv
Total factors: 7793

tipo_aggiudicatario
Total factors: 7


## Select a model

First, we *dummify* the categorical variables

In [260]:
#onehotencoder = pd.get_dummies(training_df['settore'], drop_first=True)
#onehotencoder

In [261]:
#onehotencoder = pd.get_dummies(training_df['tipo_scelta_contraente'], drop_first=True)
#onehotencoder

In [262]:
# Drop column non_numeric_col as it is now encoded
#X = X.drop('Gender',axis = 1)
# Join the encoded dataframe to X
#X = X.join(onehotencoder)

In [263]:
#test_df

## KNN Classifier

In [264]:
training_df = training_df.reset_index()

training_df

,level_0,index,cig,numero_gara,importo_complessivo_gara,n_lotti_componenti,importo_lotto,settore,data_pubblicazione,tipo_scelta_contraente,modalita_realizzazione,denominazione_amministrazione_appaltante,sezione_regionale,descrizione_cpv,aggiudicatario,tipo_aggiudicatario
0,0,1396703,8103641819,7599631,876344.28,4.0,220888.00,SETTORI ORDINARI,2020-01-27,PROCEDURA APERTA,CONTRATTO D'APPALTO,CITTA METROPOLITANA DI CATANIA,SEZIONE REGIONALE SICILIA,SERVIZI DI ISPEZIONE TECNICA DI EDIFICI,A.T.I. SLURP,"IMPRESA SINGOLA (IMPRENDITORI INDIVIDUALI, ANC..."
1,1,1396704,809634968D,7593971,140136.00,1.0,140136.00,SETTORI SPECIALI,2020-01-03,AFFIDAMENTO DIRETTO IN ADESIONE AD ACCORDO QUA...,CONTRATTO D'APPALTO DISCENDENTE DA ACCORDO QUA...,RETE FERROVIARIA ITALIANA - SOCIETA PER AZIONI,SEZIONE REGIONALE CENTRALE,INSTALLAZIONE DI DISPOSITIVI DI TELECOMUNICAZIONE,GENERALE COSTRUZIONI FERROVIARIE SPA,ATI (RAGGRUPPAMENTI TEMPORANEI DI CONCORRENTI:...
2,2,1396705,809634968D,7593971,140136.00,1.0,140136.00,SETTORI SPECIALI,2020-01-03,AFFIDAMENTO DIRETTO IN ADESIONE AD ACCORDO QUA...,CONTRATTO D'APPALTO DISCENDENTE DA ACCORDO QUA...,RETE FERROVIARIA ITALIANA - SOCIETA PER AZIONI,SEZIONE REGIONALE CENTRALE,INSTALLAZIONE DI DISPOSITIVI DI TELECOMUNICAZIONE,VALTELLINA SPA,ATI (RAGGRUPPAMENTI TEMPORANEI DI CONCORRENTI:...
3,3,1396706,81621757E2,7649364,380000.00,1.0,380000.00,SETTORI ORDINARI,2020-01-02,AFFIDAMENTO DIRETTO IN ADESIONE AD ACCORDO QUA...,CONTRATTO DAPPALTO DISCENDENTE DA ACCORDO QUAD...,AMMINISTRAZIONE PROVINCIALE DI TERNI,SEZIONE REGIONALE UMBRIA,ELETTRICITÀ,A2A ENERGIA SPA,"IMPRESA SINGOLA (IMPRENDITORI INDIVIDUALI, ANC..."
4,4,1396707,8142536944,7633063,712031.30,1.0,712031.30,SETTORI ORDINARI,2020-01-13,PROCEDURA NEGOZIATA SENZA PREVIA PUBBLICAZIONE,CONTRATTO DAPPALTO,AZIENDA ULSS N. 3 SERENISSIMA,SEZIONE REGIONALE VENETO,SERVIZI DI MANUTENZIONE DI SISTEMI E DI ASSIST...,DEDALUS,"IMPRESA SINGOLA (IMPRENDITORI INDIVIDUALI, ANC..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110967,110967,1507670,85286619B2,7958292,60638.48,1.0,60638.48,SETTORI ORDINARI,2020-11-23,AFFIDAMENTO DIRETTO,CONTRATTO DAPPALTO,COMUNE DI MONTEROSSO AL MARE,SEZIONE REGIONALE LIGURIA,SERVIZI DI INGEGNERIA INTEGRATI,OPPO VALERIO,ATI (RAGGRUPPAMENTI TEMPORANEI DI CONCORRENTI:...
110968,110968,1507671,8498088C0E,7932065,44000.00,1.0,44000.00,SETTORI SPECIALI,2020-11-20,AFFIDAMENTO DIRETTO IN ADESIONE AD ACCORDO QUA...,CONTRATTO DAPPALTO DISCENDENTE DA ACCORDO QUAD...,RETE FERROVIARIA ITALIANA - SOCIETA PER AZIONI,SEZIONE REGIONALE CENTRALE,"RIPARAZIONE, MANUTENZIONE E SERVIZI AFFINI CON...",ECM SPA,"IMPRESA SINGOLA (IMPRENDITORI INDIVIDUALI, ANC..."
110969,110969,1507672,8489155053,7924202,60000.00,1.0,60000.00,SETTORI ORDINARI,2020-11-02,PROCEDURA NEGOZIATA SENZA PREVIA PUBBLICAZIONE,CONTRATTO DAPPALTO,AZIENDA UNITA SANITARIA LOCALE N. 3 GENOVESE,SEZIONE REGIONALE LIGURIA,DISPOSITIVI E PRODOTTI MEDICI VARI,DIMAR,"IMPRESA SINGOLA (IMPRENDITORI INDIVIDUALI, ANC..."
110970,110970,1507673,8523381482,7953319,131903.00,1.0,131903.00,SETTORI ORDINARI,2020-11-30,PROCEDURA APERTA,CONTRATTO DAPPALTO,S.E.A. - SERVIZI E AMBIENTE S.P.A.,SEZIONE REGIONALE MOLISE,"GESTIONE DEL PARCO MACCHINE, SERVIZI DI RIPARA...",AUTORIPARAZIONI VIMA SAS,"IMPRESA SINGOLA (IMPRENDITORI INDIVIDUALI, ANC..."


In [266]:
tr_input = prepareInputForKNN(df)

knn_classifier = KNeighborsClassifier(100)
knn_classifier.fit(tr_input[0], tr_input[1])

KNeighborsClassifier(n_neighbors=100)

In [267]:
ts_input = prepareInputForKNN(test_df)

true_output = ts_input[1]
pred_output = KNN_model.predict(ts_input[0])

In [268]:
matches = 0
total = len(pred_output)
for i in range(total):
    if true_output[i] == pred_output[i]:
        matches = matches+1

print("Matches:", matches)
print("Total:",total)

Matches: 38
Total: 7177
